 Using SMOTE to oversample the classification target variable

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer
from collections import Counter
from imblearn.over_sampling import SMOTE, ADASYN

ModuleNotFoundError: No module named 'imblearn'